In [1]:
import pandas as pd


In [3]:
data = pd.read_excel('transaction.xlsx')
data

,Unnamed: 0,amount,a/c_number,trans_type,description,reference_number,trans_branch,datetime
0,0,200.00,070******357,Debited,AIRTIME/ MTN/08169327250,099MJKL222600FUo,HEAD OFFICE,2022-09-17 04:45:42
1,1,300.00,070******357,Debited,AIRTIME/ MTN/08168550974,099MJKL22258SUsT,HEAD OFFICE,2022-09-16 03:20:03
2,2,500.00,070******357,Debited,TRF/From TelRich Services/FRM GOODRICH IFE,099MJKL22258LPM0,HEAD OFFICE,2022-09-15 23:29:28
3,3,200.00,070******357,Debited,AIRTIME/ MTN/08169327250,099MJKL2225802x3,HEAD OFFICE,2022-09-15 05:07:19
4,4,300.00,070******357,Debited,AIRTIME/ MTN/08168550974,099MJKL22257SR8s,HEAD OFFICE,2022-09-15 03:08:11
...,...,...,...,...,...,...,...,...
495,495,1600.00,070******357,Credited,JOSEPH EPHRAIM MARK/PAYMENT,099MNIP213433oEZ,HEAD OFFICE,2021-12-09 18:10:53
496,496,2510.75,070******357,Debited,TRF/From Emem Victor/FRM GOODRICH IFEANYI,099MJKL213438671,HEAD OFFICE,2021-12-09 16:29:36
497,497,3010.75,070******357,Debited,TRF/From Patience/FRM GOODRICH IFEANYI OKO,099MJKL213437Lsq,HEAD OFFICE,2021-12-09 15:59:13
498,498,2010.75,070******357,Debited,TRF//FRM GOODRICH IFEANYI OKORO TO CHRISTO,099MJKL213435ULR,HEAD OFFICE,2021-12-09 14:51:32
